# ToolBench Cache Review

In [14]:
import json, jsonlines
from tqdm import tqdm

In [16]:
def compare_dict(dict1, dict2):
    if dict1.keys() != dict2.keys():
        return False
    for key in dict1.keys():
        if dict1[key] != dict2[key]:
            return False
    return True

In [3]:
cache_file = "tmp/toolbench_cache.jsonl"

In [25]:
#json_cache_data = list(jsonlines.open(cache_file))
json_cache_data = [json.loads(line.strip()) for line in open(cache_file).readlines()]
# json_cache_data[:3]
json_cache_data[83]

{'key': 'Data.keyword_analysis.querykeywords.truncate',
 'value': {'tool_input': '{\n  "q": "birthday party ideas"\n\n',
  'status_code': 0,
  'response': '{"error": "Tool input parse error...\\n", "response": ""}'}}

In [29]:
all_callings = []
total_errors = 0
for idx, data in enumerate(tqdm(json_cache_data)):
    api_key = data['key']
    category, api_name, api_func, strip_method = api_key.split(".")
    value = data['value']
    status_code = value['status_code']
    has_error = False
    try:
        tool_input = json.loads(value['tool_input'])
    except Exception as e:
        has_error = True
        print("--------------------------------------")
        # print(f"{data=}")
        # print(f"{value=}")
        print(f"{idx=},{status_code=}\ntool_input: {value['tool_input']}\nExcept {e=}")
        tool_input = value['tool_input']
    response = value['response']
    try:
        rsp_data = json.loads(response)
    except Exception as e:
        has_error = True
        print("--------------------------------------")
        print(f"{idx=},{status_code=}\nresponse: {response=}\nException: {e=}")
        rsp_data = response

    if has_error:
        total_errors += 1
    calling = {
        'category': category,
        'api_name': api_name,
        'api_func': api_func,
        'strip_method': strip_method,
        'tool_input': tool_input,
        'status_code': status_code,
        'response': rsp_data,        
    }
    all_callings.append(calling)
print(f"{total_errors=}")
all_callings[:3]
    

100%|██████████| 744/744 [00:00<00:00, 170215.58it/s]

--------------------------------------
idx=83,status_code=0
tool_input: {
  "q": "birthday party ideas"


Except e=JSONDecodeError("Expecting ',' delimiter: line 4 column 1 (char 33)")
--------------------------------------
idx=108,status_code=0
tool_input: {
Function: {"error": "", "response": "[{'countryName': 'Egypt', 'countryLogo': 'https://cdn.sportmonks.com/images/countries/png/short/eg.png', 'leagueName': 'Premier League', 'data': None, 'localTeamName': 'Future FC', 'localTeamLogo': 'https://cdn.sportmonks.com/images/soccer/teams/6/250630.png', 'visitorTeamName': 'Al Ahly', 'visitorTeamLogo': 'https://cdn.sportmonks.com/images/soccer/teams/12/460.png', 'bet': '1', 'betQuote': 1.57, 'probability': 83.33, 'matchDate': '2023-07-02T17:30:00', 'betState': 0, 'hits': 4}]"}
}
Except e=JSONDecodeError('Expecting property name enclosed in double quotes: line 2 column 1 (char 2)')
--------------------------------------
idx=118,status_code=0
tool_input: {
  "points": 12000

Function: {"err

[{'category': 'Travel',
  'api_name': 'flight_fare_search',
  'api_func': 'airport_search',
  'strip_method': 'truncate',
  'tool_input': {'query': 'London'},
  'status_code': 0,
  'response': {'error': '',
   'response': "{'query': 'London', 'result': [{'iata': 'LHR', 'icao': 'EGLL', 'airportName': 'London Heathrow Airport', 'airportCity': 'London, EN, GB', 'airportCityCode': 'LON', 'airportCountryCode': 'GB'}, {'iata': 'LGW', 'icao': 'EGKK', 'airportName': 'London Gatwick Airport', 'airportCity': 'London, EN, GB', 'airportCityCode': 'LON', 'airportCountryCode': 'GB'}, {'iata': '', 'icao': 'EGLW', 'airportName': 'London Heliport', 'airportCity': 'London, GB', 'airportCityCode': '', 'airportCountryCode': 'GB'}, {'iata': 'QQP', 'icao': '', 'airportName': 'Paddington Railway Station', 'airportCity': 'London, GB', 'airportCityCode': 'LON', 'airportCountryCode': 'GB'}, {'iata': 'GON', 'icao': 'KGON', 'airportName': 'Groton-New London Airport', 'airportCity': 'New London, CT, US', 'airportC

In [ ]:

# status_code: 
# 1 Hallucinating function names
# 4 means that the model decides to pruning by itself
# 5 represents api call timeout
# 6 for 404
# 7 means not subscribed
# 8 represents unauthorized
# 9 represents too many requests
# 10 stands for rate limit
# 11 message contains "error" field
# 12 error sending request

In [21]:
all_callings[83]

{'category': 'Data',
 'api_name': 'keyword_analysis',
 'api_func': 'querykeywords',
 'strip_method': 'truncate',
 'tool_input': '{\n  "q": "birthday party ideas"\n\n',
 'status_code': 0,
 'response': {'error': 'Tool input parse error...\n', 'response': ''}}